In [1]:
from asset_tracker.models import load_spatialite_sqlite_extension
from sqlalchemy import create_engine

database_engine = create_engine('sqlite://', echo=True)
load_spatialite_sqlite_extension(database_engine)

2020-02-07 04:09:50,753 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-02-07 04:09:50,754 INFO sqlalchemy.engine.base.Engine ()
2020-02-07 04:09:50,756 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-02-07 04:09:50,757 INFO sqlalchemy.engine.base.Engine ()
2020-02-07 04:09:50,766 INFO sqlalchemy.engine.base.Engine SELECT InitSpatialMetaData() AS "InitSpatialMetaData_1"
2020-02-07 04:09:50,766 INFO sqlalchemy.engine.base.Engine ()


Engine(sqlite://)

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

get_database_session = sessionmaker()
get_database_session.configure(bind=database_engine)
db = get_database_session()

In [3]:
from asset_tracker.models.meta import Base
Base.metadata.create_all(database_engine)

2020-02-07 04:09:50,876 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("asset")
2020-02-07 04:09:50,879 INFO sqlalchemy.engine.base.Engine ()
2020-02-07 04:09:50,883 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("bus")
2020-02-07 04:09:50,885 INFO sqlalchemy.engine.base.Engine ()
2020-02-07 04:09:50,886 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("connection")
2020-02-07 04:09:50,888 INFO sqlalchemy.engine.base.Engine ()
2020-02-07 04:09:50,890 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("line_type")
2020-02-07 04:09:50,891 INFO sqlalchemy.engine.base.Engine ()
2020-02-07 04:09:50,894 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE asset (
	id VARCHAR NOT NULL, 
	creation_datetime DATETIME, 
	modification_datetime DATETIME, 
	_attributes BLOB, 
	name VARCHAR, 
	type_code VARCHAR(11), 
	CONSTRAINT pk_asset PRIMARY KEY (id), 
	CONSTRAINT ck_asset_assettypecode CHECK (type_code IN ('LINE', 'METER', 'TRANSFORMER', 'SUBSTATION'))
)


2020-02-07 04:09:50,897 I

In [4]:
from asset_tracker.models import Asset, Bus, Connection

In [5]:
asset = Asset.make_unique_record(db)

2020-02-07 04:09:50,971 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-02-07 04:09:50,975 INFO sqlalchemy.engine.base.Engine INSERT INTO asset (id, creation_datetime, modification_datetime, _attributes, name, type_code, geometry) VALUES (?, ?, ?, ?, ?, ?, GeomFromEWKT(?))
2020-02-07 04:09:50,977 INFO sqlalchemy.engine.base.Engine ('avqMeW3n', '2020-02-07 09:09:50.975622', None, None, None, None, None)


In [6]:
bus1 = Bus.make_unique_record(db)

2020-02-07 04:09:50,990 INFO sqlalchemy.engine.base.Engine INSERT INTO bus (id) VALUES (?)
2020-02-07 04:09:50,991 INFO sqlalchemy.engine.base.Engine ('Jn37tLou',)


In [7]:
bus2 = Bus.make_unique_record(db)

2020-02-07 04:09:51,006 INFO sqlalchemy.engine.base.Engine INSERT INTO bus (id) VALUES (?)
2020-02-07 04:09:51,007 INFO sqlalchemy.engine.base.Engine ('SabXrJO4',)


In [8]:
asset.connections

2020-02-07 04:09:51,025 INFO sqlalchemy.engine.base.Engine SELECT connection._attributes AS connection__attributes, connection.asset_id AS connection_asset_id, connection.bus_id AS connection_bus_id 
FROM connection 
WHERE ? = connection.asset_id
2020-02-07 04:09:51,026 INFO sqlalchemy.engine.base.Engine ('avqMeW3n',)


[]

In [9]:
# connection1 = Connection(asset_id=asset.id, bus_id=bus1.id, attributes=None)
# connection2 = Connection(asset_id=asset.id, bus_id=bus2.id, attributes={})

connection1 = Connection(bus_id=bus1.id, attributes={})
connection2 = Connection(bus_id=bus2.id)
connection2.attributes = {}

hey
hey


In [10]:
asset.connections = [connection1, connection2]

In [11]:
db.add(asset)

In [12]:
db.commit()

2020-02-07 04:09:51,080 INFO sqlalchemy.engine.base.Engine INSERT INTO connection (_attributes, asset_id, bus_id) VALUES (?, ?, ?)
2020-02-07 04:09:51,082 INFO sqlalchemy.engine.base.Engine ((None, 'avqMeW3n', 'Jn37tLou'), (None, 'avqMeW3n', 'SabXrJO4'))
2020-02-07 04:09:51,084 INFO sqlalchemy.engine.base.Engine COMMIT


In [13]:
asset.connections

2020-02-07 04:07:26,872 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-02-07 04:07:26,876 INFO sqlalchemy.engine.base.Engine SELECT asset.id AS asset_id, asset.creation_datetime AS asset_creation_datetime, asset.modification_datetime AS asset_modification_datetime, asset._attributes AS asset__attributes, asset.name AS asset_name, asset.type_code AS asset_type_code, AsEWKB(asset.geometry) AS asset_geometry 
FROM asset 
WHERE asset.id = ?
2020-02-07 04:07:26,877 INFO sqlalchemy.engine.base.Engine ('kSFkyysT',)
2020-02-07 04:07:26,880 INFO sqlalchemy.engine.base.Engine SELECT connection._attributes AS connection__attributes, connection.asset_id AS connection_asset_id, connection.bus_id AS connection_bus_id 
FROM connection 
WHERE ? = connection.asset_id
2020-02-07 04:07:26,882 INFO sqlalchemy.engine.base.Engine ('kSFkyysT',)


[<Connection(asset_id=kSFkyysT, bus_id=PjjMouW6)>,
 <Connection(asset_id=kSFkyysT, bus_id=fN4dh7pB)>]

In [14]:
asset.connections = [connection1]

In [42]:
db.add(asset)

In [43]:
db.commit()

2020-02-07 03:54:06,762 INFO sqlalchemy.engine.base.Engine DELETE FROM connection WHERE connection.asset_id = ? AND connection.bus_id = ?
2020-02-07 03:54:06,763 INFO sqlalchemy.engine.base.Engine ('7eCbSfSa', 'D49kxoph')
2020-02-07 03:54:06,765 INFO sqlalchemy.engine.base.Engine COMMIT
